# spaCy

spaCy es otra librería open source en Python para NLP.

La diferencia fundamental entre NLTK y spaCy es que el primero es muy cómodo para aprender e iniciarse mientras que el segundo está pensado para productizar.

Gensim, otra librería, la veremos más adelante cuando estudiemos Topic Modeling y Word Embeddings.

Documentación de spaCy: https://spacy.io/

La filosofía de trabajo en spaCy es que si existen una serie de algoritmos que solucionan un problema, dar la solución al problema con un único. Además, su funcionamiento se basa en la construcción de pipelines.


<img src=https://i.imgur.com/nD7ut2U.jpg>

¿Qué capacidades (modelos) linguísticas nos ofrece spaCy?

<img src=https://i.imgur.com/lGcL6lx.jpg>

Es decir, de spaCy podremos sacar siempre que queramos tokens, pos tags, árboles de dependencia, o entidades nombradas. Incluye también modelos de word embeddings (que veremos con más detalle en sesiones posteriores).

<img src=https://spacy.io/architecture-bcdfffe5c0b9f221a2f6607f96ca0e4a.svg width=550px>

## Modelos de spaCy

Modelos pre-entrenados para diferentes idiomas y con diferentes corpus. Pueden ser descargados de diferentes maneras, tanto descarga directa, como con pip.

Link: https://spacy.io/usage/models



In [ ]:
# !python -m spacy download es_core_news_sm
# !python -m spacy download en_core_web_sm
# !python -m spacy download en

In [ ]:
# pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.0.0/en_core_web_sm-2.0.0.tar.gz --no-deps
# pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-2.3.0/es_core_news_sm-2.3.0.tar.gz#egg=es_core_news_sm==2.3.0 --no-deps

In [ ]:
# !pip install -U spacy download es_core_news_sm
# !pip install -U spacy download en_core_web_sm

In [ ]:
# import spacy
# import es_core_news_sm
# import en_core_web_sm

# nlp_es = es_core_news_sm.load()
# nlp_en = en_core_web_sm.load()

In [ ]:
import spacy

# Modelo "pequeño" entrenado con noticias en castellano
# https://spacy.io/models/es
nlp_es = spacy.load('es_core_news_sm')

# Modelo "pequeño" entrenado con página
# https://spacy.io/models/en
nlp_en = spacy.load('en_core_web_sm')

In [ ]:
# Pipeline del modelo por defecto
nlp_es.pipeline

Pipelines en spaCy:
https://spacy.io/usage/processing-pipelines

<img src=https://d33wubrfki0l68.cloudfront.net/16b2ccafeefd6d547171afa23f9ac62f159e353d/48b91/pipeline-7a14d4edd18f3edfee8f34393bff2992.svg width=700px>

In [27]:
text = 'Mi nombre es Carlos y vivo en Madrid. Hoy es martes 8 de junio'
doc = nlp_es(text)

In [28]:
print(doc)

Mi nombre es Carlos y vivo en Madrid. Hoy es martes 8 de junio


# Tokenizing

## Frases

In [29]:
for idx, sent in enumerate(doc.sents):
    print('Frase {0:5}{1:5}'.format(str(idx), sent.text))

Frase 0    Mi nombre es Carlos y vivo en Madrid.
Frase 1    Hoy  
Frase 2    es martes 8 de junio


## Tokens

In [30]:
for idx, token in enumerate(doc):
    print('Token {0:5}{1:5}'.format(str(idx), token.text))

Token 0    Mi   
Token 1    nombre
Token 2    es   
Token 3    Carlos
Token 4    y    
Token 5    vivo 
Token 6    en   
Token 7    Madrid
Token 8    .    
Token 9    Hoy  
Token 10   es   
Token 11   martes
Token 12   8    
Token 13   de   
Token 14   junio


In [32]:
print('{0:10}{1:10}{2:5}'.format('Token', 'Shape', 'is_alpha'))
for token in doc:
    print('{0:10}{1:10}{2:5}'.format(token.text, token.shape_, str(token.is_alpha)))

Token     Shape     is_alpha
Mi        Xx        True 
nombre    xxxx      True 
es        xx        True 
Carlos    Xxxxx     True 
y         x         True 
vivo      xxxx      True 
en        xx        True 
Madrid    Xxxxx     True 
.         .         False
Hoy       Xxx       True 
es        xx        True 
martes    xxxx      True 
8         d         False
de        xx        True 
junio     xxxx      True 


# Normalización de texto

In [38]:
# Eliminar stop words
from spacy.lang.es.stop_words import STOP_WORDS

print(list(STOP_WORDS)[:20])

['tiene', 'tuyo', 'unos', 'lugar', 'vaya', 'general', 'ningún', 'ésos', 'pueda', 'usted', 'alguna', 'raras', 'sean', 'detrás', 'éstos', 'mío', 'buenas', 'he', 'entre', 'quienes']


In [39]:
ex_text_1 = 'Soy una frase de ejemplo de la cual vamos a eliminar los stopwords'
[word for word in ex_text_1.lower().split() if word not in STOP_WORDS]

['frase', 'a', 'eliminar', 'stopwords']

Debate: ¿Pensáis que, en general, se deben filtrar los stopwords?

In [40]:
ex_text_2 = 'No me gusta esta canción'
[word for word in ex_text_2.lower().split() if word not in STOP_WORDS]

['gusta', 'canción']

# Part of Speech tagging

El PoS Tagging es una técnica **fundamental** en NLP que consiste en etiquetar cada palabra de un documento en su correspondiente categría gramatical.

<img src=https://blog.aaronccwong.com/assets/images/bigram-hmm/pos-title.jpg width=650px>

¿Utilidad? Muchísima:

- Posibilidad de encontrar los adjetivos / sustantivos /adverbios más comunes
- _Ayuda_ a Lemmatizers al desambiguar entre palabras
- Grafos en los que los nodos son entidades y verbos. Posibilidad de analizar relaciones entre entidades -> Pintar ejemplo
- Posibles features (la distribución de categorías no siempre es homogénea en función del contexto)

Podríamos hacer un algoritmo que mirara verbos entre otras entidades, como nombres o adjetivos, y que fuera el verbo quien decidiera que relación tienen esas entidades. Eso se suele hacer para crear bases de datos de grafos, donde cada nodo es una entidad, y entre entidades hay relaciones.

Veamos algún ejemplo:

Atributos (https://spacy.io/api/token#attributes):
- pos_: tipo de palabra (sustantivo, verbo, adjetivo, etc)
- tag_: tipo de palabra especificando más atributos
- dep_: relación de dependencia sintáctica

Explicación de los términos:
https://github.com/explosion/spaCy/blob/master/spacy/glossary.py

In [41]:
doc

Mi nombre es Carlos y vivo en Madrid. Hoy es martes 8 de junio

In [42]:
print('{0:10}{1:10}'.format('Token', 'pos'))
for idx, token in enumerate(doc):
    print('{0:10}{1:10}'.format(token.text, token.pos_))

Token     pos       
Mi        DET       
nombre    NOUN      
es        AUX       
Carlos    PROPN     
y         CCONJ     
vivo      VERB      
en        ADP       
Madrid    PROPN     
.         PUNCT     
Hoy       ADV       
es        AUX       
martes    NOUN      
8         NUM       
de        ADP       
junio     NOUN      


In [43]:
print('{0:10}{1:10}'.format('Token', 'tag'))
for idx, token in enumerate(doc):
    print('{0:10}{1:10}'.format(token.text, token.tag_))

Token     tag       
Mi        DET       
nombre    NOUN      
es        AUX       
Carlos    PROPN     
y         CCONJ     
vivo      VERB      
en        ADP       
Madrid    PROPN     
.         PUNCT     
Hoy       ADV       
es        AUX       
martes    NOUN      
8         NUM       
de        ADP       
junio     NOUN      


In [44]:
print('{0:10}{1:10}{2:10}'.format('Token', 'dep', 'Meaning'))
for idx, token in enumerate(doc):
    print('{0:10}{1:10}{2:10}'.format(token.text, token.dep_, str(spacy.explain(token.dep_))))

Token     dep       Meaning   
Mi        det       determiner
nombre    nsubj     nominal subject
es        cop       copula    
Carlos    ROOT      None      
y         cc        coordinating conjunction
vivo      conj      conjunct  
en        case      case marking
Madrid    nmod      modifier of nominal
.         punct     punctuation
Hoy       ROOT      None      
es        cop       copula    
martes    ROOT      None      
8         compound  compound  
de        case      case marking
junio     compound  compound  


# Dependencia sintáctica

In [45]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True, options={'distance':100})

# Reconocimiento de entidades nombradas (NER)

El reconocimiento de entidades nombradas (Named Entity Recognition, NER, por sus siglas en inglés) trata de detectar posibles entidades nombradas y, posteriormente, clasificarlas entre un conjunto de categorías predefinidas.

Ejemplos de entidades nombradas: nombres de personas, lugares, cantidades, empresas...

Pero, ¿qué es exactamente una _entidad nombrada_? Según definió Saul Kripke * (filósofo y lógico) son - o deberían ser - todas aquellas entidades para las cuales existe uno - o más de uno - designador rígido. Es decir, dicha palabra / expresión se refiere a la misma cosa / entidad con independencia del contexto.

<img src=https://hyscore.io/wp-content/uploads/2019/03/illustration_named_entity_recognition-1024x486-1.jpg width=700px>

El rendimiento de los NER varía mucho en función del idioma en el que han sido entrenados. El rendimiento que se comienza a obtener (debido principalmente al uso de modelos de embeddings contextuales) supera al de un ser humano.

Un enlace intersante: https://primer.ai/blog/a-new-state-of-the-art-for-named-entity-recognition/

* _El nombrar y la necesidad_ (Saul Kripke), 1980

In [46]:
doc_ner_1 = nlp_es('Jim compró 300 acciones de Acme Corp. en 2006')
displacy.render(doc_ner_1, style='ent', jupyter=True, options={'distance':100})

In [48]:
doc_ner_2 = nlp_en('Peter bought 300 shares of Acme Corp. in 2006')
displacy.render(doc_ner_2, style='ent', jupyter=True, options={'distance':100})

In [49]:
doc_ner_2 = nlp_en('I heard that Paris Hilton stayed at the Hilton in Paris')
displacy.render(doc_ner_2, style='ent', jupyter=True, options={'distance':100})

In [50]:
print('{0:10}{1:10}'.format('Token', 'Entity Label'))
for entity in doc_ner_1.ents:
    print('{0:10}{1:10}'.format(entity.text, entity.label_))

Token     Entity Label
Jim       PER       
Acme Corp PER       


In [51]:
print('{0:10}{1:10}'.format('Token', 'Entity Label'))
for entity in doc_ner_2.ents:
    print('{0:10}{1:10}'.format(entity.text, entity.label_))

Token     Entity Label
Paris     GPE       
Hilton    GPE       
Hilton    GPE       
Paris     GPE       


# Lemmatization

Técnica de normalización de textos que busca reducir las palabras a su raíz (lemma).

Muy utilizado para reducir la cardinalidad del vocabulario asociando para diferentes formas flexionadas un único token ('entreno', 'entrenarás', 'entrenaría' -> 'entrenar').

Aunque muy utilizados en motores de búsqueda 

In [ ]:
print('{0:10}{1:10}{2:10}'.format('Token', 'Lemma', 'PoS Tag'))
for idx, token in enumerate(doc):
    print('{0:10}{1:10}{2:10}'.format(token.text, token.lemma_, token.pos_))

In [54]:
text_2 = 'comer comiendo comieron comedor comeré comerá comerás'
text_3 = 'comerás'
doc_2 = nlp_es(text_2)
doc_3 = nlp_es(text_3)


print('Text 2')
print('{0:10}{1:10}{2:10}'.format('Token', 'Lemma', 'PoS Tag'))
for idx, token in enumerate(doc_2):
    print('{0:10}{1:10}{2:10}'.format(token.text, token.lemma_, token.pos_))

print('\nText 3')
print('{0:10}{1:10}{2:10}'.format('Token', 'Lemma', 'PoS Tag'))
for idx, token in enumerate(doc_3):
    print('{0:10}{1:10}{2:10}'.format(token.text, token.lemma_, token.pos_))

Text 2
Token     Lemma     PoS Tag   
comer     comer     VERB      
comiendo  comer     VERB      
comieron  comer     VERB      
comedor   comedor   NOUN      
comeré    comeré    ADJ       
comerá    comer     VERB      
comerás   comerás   ADP       

Text 3
Token     Lemma     PoS Tag   
comerás   comerás   VERB      


# Lemmatization vs Stemming

Ambas tienen como objetivo reducir las palabras a su raíz léxica.

- **Lemmatization**:

    Tiene en consideración el análisis morfológico de las palabras. Son necesarios diccionarios completos de formas flexionadas y raíces (lemmas).
    
    A veces es necesario desambiguar. P. ej.: "planta" (planta vs plantar)

<img src=https://blog.bitext.com/hs-fs/hubfs/lemma_v2.png width=500px>

- **Stemming**:
    
    Algoritmos que mediante heurísticos / reglas tratan de reducir las palabras a una posible raíz (stem) mediante la eliminación de algunos prefijos y sufijos. Más sencillo que un lemmatizer
    
    No hay garantía de que el resultado sea una palabra real.
    
    El algoritmo más utilizado en Inglés es el de Porter que consiste en 5 fases de reducción de la palabra aplicadas de manera secuencial.
    <img src=https://blog.bitext.com/hs-fs/hubfs/stemming_v2.png width=250px>

# Similitud

Si trabajamos con los _small models_ (SM) en lugar de los _large models_ (LM), en lugar de trabajar con _word vectors_ estaremos trabajando con _context-sensitive tensors_.

No os preocupéis, los modelos de _word embeddings_ los estudiaremos con mucho más detalle en próximas sesiones :D

Como adelanto, en el caso de trabajar con los primeros (word vectors) siempre tendremos el mismo vector para el mismo token. Si trabajamos con los segundos (context-sensitive tensors) el vector del token estará determinado por su contexto (el resto de tokens que le rodean).

Aunque el concepto de similitud lo veremos más adelante, como adelanto, conocer que con spaCy es posible calcular lo _parecidos_ o _distintos_ que son dos tokens.

In [ ]:
# pip3 install https://github.com/explosion/spacy-models/releases/download/es_core_news_md-2.3.0/es_core_news_md-2.3.0.tar.gz#egg=es_core_news_md==2.3.0 --no-deps

In [ ]:
# !python3 -m spacy download es_core_news_md

In [55]:
# !python3 -m spacy download es_core_news_md
import spacy
nlp_es_md = spacy.load('es_core_news_md')

In [56]:
word_1 = nlp_es_md('verde')
word_2 = nlp_es_md('azul')
word_3 = nlp_es_md('mariposa')

print('Similarity between word {} and word {}: {:0.6f}'.format(1, 2, word_1.similarity(word_2)))
print('Similarity between word {} and word {}: {:0.6f}'.format(1, 3, word_1.similarity(word_3)))
print('Similarity between word {} and word {}: {:0.6f}'.format(2, 3, word_2.similarity(word_3)))

Similarity between word 1 and word 2: 0.743638
Similarity between word 1 and word 3: 0.382572
Similarity between word 2 and word 3: 0.442991


In [59]:
sent_1 = nlp_es_md('me gusta el color verde')
sent_2 = nlp_es_md('me gusta el azul')
sent_3 = nlp_es_md('me gusta la mariposa')

print('Similarity between sent {} and sent {}: {:0.6f}'.format(1, 2, sent_1.similarity(sent_2)))
print('Similarity between sent {} and sent {}: {:0.6f}'.format(1, 3, sent_1.similarity(sent_3)))
print('Similarity between sent {} and sent {}: {:0.6f}'.format(2, 3, sent_2.similarity(sent_3)))

Similarity between sent 1 and sent 2: 0.967595
Similarity between sent 1 and sent 3: 0.651385
Similarity between sent 2 and sent 3: 0.653034


# El Universo de spaCy

Diferentes recursos (paquetes, plugins, extensiones, etc) desarrollados por o para spaCy.

https://spacy.io/universe